In [1]:
import pandas as pd

In [2]:
queryData = pd.read_csv('gs://data_tql/kaggle-dbqa/processed/queryDataKaggleDBQA.csv')
tableData = pd.read_csv('gs://data_tql/kaggle-dbqa/processed/tablesSchemaKaggleDBQA.csv')

In [3]:
def create_schema_natural_language(row):

    schema_id = row['schema_id']
    table_name = row['table_name']
    primary_key = row['primary_key']
    column_list = eval(row['column_list_original'])
    datatype_list = eval(row['column_datatypes'])
    foreign_key = eval(row['foreign_keys'])

    column_list_with_datatype = []
    for column, datatype in zip(column_list, datatype_list):
        column_list_with_datatype.append(' has datatype '.join([column, datatype]))

    schema_natural_language = f"Given the Table {table_name} having columns as {', '.join(column_list_with_datatype)} which has {primary_key}"
    return schema_natural_language

tableData['schema_natural_language'] = tableData.apply(create_schema_natural_language, axis = 1)

all_schemas = tableData['schema_id'].unique()
schema_table_query = {}
for schema in all_schemas:
    schema_details = ' and '.join(tableData[tableData['schema_id'] == schema]['schema_natural_language'].values)
    schema_table_query[schema] = schema_details

instruction = "Answer the question by writing the appropriate SQL code."    
queryData['schema_natural_language'] = queryData['db_id'].map(schema_table_query)
queryData['question'] = queryData['schema_natural_language'] + '. ' + instruction + ' ' + queryData['TQL']

In [4]:
training_data = queryData[['SQL','question']]
training_data.to_csv('gs://data_tql/kaggle-dbqa/processed/kaggleDbqaTrainingData.csv', index= False)